# Cleaning The Raw Data

## Import Depnedencies

In [1]:
import pandas as pd
import numpy as np
import os, sys

## Columns, Filenames, and Directories
Some variables which describe the column names/types, filenames and input/output directories 

In [2]:
# column names for datafiles
headers=['id', 'pricing', 'datetime',
         'KwH/hh', 'acorn', 'acorn-grouped']

# column types for datafile
dtypes={'KwH/hh': 'float'}


# format-string for datafile path
pathfmt=os.path.join('.', '{dir}','{year}-{month:02}-power-survey-london.csv{suffix}')

# directories
input_dir=os.path.join('..', 'raw')
output_dir=os.path.join('..', 'data')


These next variables select the year and month we are going to load and clean; since these files are ~500MB uncompressed, we only load them one at a time. *(The script which does all of this for every relavent year and month is located in the "scripts" directory of the repository.)*

In [3]:
# year to load
year = 2012
# month (1 to 12) of the year to load 
month = 7

## Create Output Directory (if it doesn't exist)

In [4]:
try: # try to make output directory
    os.mkdir(output_dir)
except FileExistsError: # if it exists, do nothing.
    pass

## Generate Input/Output Filenames


In [5]:
output_path = pathfmt.format( year=year, month=month, dir=output_dir, suffix='' )
input_path = pathfmt.format( year=year, month=month, dir=input_dir, suffix='.bz2')

print(f"Processing file {input_path}")

Processing file .\..\raw\2012-07-power-survey-london.csv.bz2


## Load Raw Data

In [6]:
data = pd.read_csv( input_path,    # file to load 
                    header=None,   # month/year files do not have a header 
                    names=headers )# names to use for the columns 

### Extract Non-Nil Records 
Only look at records with valid power usage reading; entries without valid readings have 'Null' where the power usage should be. In addition, make sure cleaned 'KwH/hh' column consists of floats.

In [7]:
tmp = data.copy()
tmp = tmp.loc[ tmp['KwH/hh'] != 'Null' ]
data = tmp.astype( dtypes ).copy()
del tmp # free up some resources

C:\Users\burned\anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


### Fix Datetime And ACORN Column

In [8]:
# Note: this next bit only works because the time format is close to 'normal'
data['datetime'] = pd.to_datetime( data['datetime'] )

# Remove "ACORN-" from each entry in the 'acorn' column; keeping only the group letter
data['acorn'] = data['acorn'].apply( lambda x : x[-1] )

# take care of possible double counting
data = data.groupby(['id', 'datetime', 'acorn', 'acorn-grouped']).mean().reset_index()

### Compute Values For Each ACORN Groups

In [9]:
# group data by datetime and acorn type
grouped = data.groupby(['datetime', 'acorn'])

# compute/combine values for each datetime and  acorn type 
sums = grouped.sum() # combine the power usage
stds = grouped.std() # compute sample standard deviation
counts = grouped.count() # count number of records for each datetime and acorn type

# merge all three values into one table
merged = pd.merge(counts['id'], sums, 
                  left_index=True, right_index=True)
merged = pd.merge(merged, stds, 
                  left_index=True, right_index=True)
merged = merged.reset_index()

### Mung Merged Data
Create columns which correspond to the ACORN type associated with the values we computed then merge results.

In [10]:
# group by datetime
grouped = merged.groupby('datetime')

# extract and mung computed values
counts_atyp = grouped.apply( lambda x : \
                          x.set_index('acorn').transpose().iloc[1])
sums_atyp = grouped.apply( lambda x : \
                         x.set_index('acorn').transpose().iloc[2])
stds_atyp = grouped.apply(lambda x : \
                         x.set_index('acorn').transpose().iloc[3])

# fixup columns names
counts_atyp = counts_atyp.rename(
    index=str,
    columns=dict([(name, name+'_count') for name in counts_atyp.columns]))
sums_atyp = sums_atyp.rename(
    index=str,
    columns=dict([(name, name+'_sigma') for name in sums_atyp.columns]))
stds_atyp = stds_atyp.rename(
    index=str,
    columns=dict([(name, name+'_std') for name in stds_atyp.columns]))

# merge everything into one dataframe
combined = pd.merge(sums_atyp,  stds_atyp,
                   left_index=True, right_index=True)
combined = pd.merge(combined, counts_atyp,
                   left_index=True, right_index=True)

### Add Summary Columns

In [11]:
count_total = counts_atyp.sum(axis=1)
sigma_total = sums_atyp.sum(axis=1)
means = sigma_total / count_total

combined['sigma'] = sigma_total
combined['count'] = count_total
combined['mean'] = means

### Preview Combined Data

In [12]:
# preview a sample from the combined data
combined.sample(5)

acorn,A_sigma,B_sigma,C_sigma,D_sigma,E_sigma,F_sigma,G_sigma,H_sigma,I_sigma,J_sigma,...,L_count,M_count,N_count,O_count,P_count,Q_count,U_count,sigma,count,mean
datetime,,,,,,,,,,,,,,,,,,,,,
2012-07-13 13:00:00,25.469,1.374,13.525,47.658,197.987,82.999,18.744,42.167,3.923,7.176,...,167,62,88,60,83,673,29,632.894,3762,0.168233
2012-07-20 03:30:00,16.039,1.256,7.739,25.404,114.935,43.999,10.508,26.310,1.740,4.060,...,167,64,88,61,83,674,29,361.774,3783,0.095632
2012-07-29 14:30:00,32.028,1.241,15.639,54.332,224.917,104.584,23.204,58.786,5.082,11.292,...,167,64,88,61,83,674,29,771.782,3784,0.203959
2012-07-23 03:30:00,16.514,0.946,7.421,25.486,116.114,45.193,11.460,25.170,1.715,4.887,...,167,63,88,61,83,673,29,367.011,3781,0.097067
2012-07-07 20:00:00,34.376,2.386,18.063,77.290,255.756,111.127,23.399,51.635,6.412,11.360,...,164,62,86,58,81,638,26,839.209,3579,0.234481


## Save Combined Data

In [13]:
print(f"Outputing to file {output_path}")

combined.to_csv(output_path)


Outputing to file .\..\data\2012-07-power-survey-london.csv
